# Monte Carlo via CPU & GPU

## CPU

In [1]:
import numpy as np

In [13]:
def calculate_asian_payoff_cpu(
    initial: float,
    r: float,
    v: float,
    dt: float,
    strike: float,
    nsim: int,
    nsteps: int,
    length: float,
) -> float:
    # intialize vectors
    St = np.zeros(shape=(nsim, nsteps+1))
    St[:, 0] = initial

    # calculate increments
    increments = calculate_increments_cpu(r, v, dt, nsim, nsteps)

    # calculate levels
    St[:, 1:] = initial * np.exp(increments.cumsum(axis=1))

    # calculate payoff at time T
    average = St.mean(axis=1)
    payoff = np.maximum(average - strike, 0.0)
    average_payoff = payoff.mean()

    # discount average payoff to time 0
    discount_factor = np.exp(-r * length)
    epv_payoff = average_payoff * discount_factor

    return epv_payoff


def calculate_increments_cpu(
    r: float,
    v: float,
    dt: float,
    nsim: int,
    nsteps: int,
):
    drift = (r - 0.5 * v * v) * dt
    diffusion = v * np.sqrt(dt)
    dWt = np.random.default_rng(1234).standard_normal((nsim, nsteps))

    increments = drift + diffusion * dWt
    return increments

## GPU 

In [3]:
import numpy as np
from numba import cuda
from math import exp, sqrt, ceil
from numba.cuda.random import xoroshiro128p_normal_float32, create_xoroshiro128p_states

In [4]:
@cuda.jit
def calculate_asian_payoff_gpu(
    initial: np.float32,
    r: np.float32,
    v: np.float32,
    dt: np.float32,
    strike: np.float32,
    nsteps: np.float32,
    length: np.int32,
    rng_states,
    epv_payoff: np.ndarray,
):
    # get index of current thread
    thread_index = cuda.grid(1) 

    # check that thread index is within bounds of output
    if thread_index < len(epv_payoff):
        # intialize variables (local memory)
        St = initial
        cumm_sum = St

        # iterate through each time step
        for _ in range(nsteps):
            # calculate next increment
            increment = calculate_increment_gpu(r, v, dt, rng_states, thread_index)

            # calculate next level
            St *= exp(increment)

            # add next level to cummulative sum
            cumm_sum += St

        # calculate average level
        average = cumm_sum / (nsteps + 1.0)

        # calculate payoff at time T
        payoff = max(average - strike, 0.0)

        # discount payoff back to time 0
        discount_factor = exp(-r * length)
        epv_payoff[thread_index] = payoff * discount_factor


@cuda.jit(device=True)
def calculate_increment_gpu(
    r: np.float32,
    v: np.float32,
    dt: np.float32,
    rng_states,
    thread_index,
) -> np.float32:
    drift = (r - 0.5 * v * v) * dt
    diffusion = v * sqrt(dt)
    dWt = xoroshiro128p_normal_float32(rng_states, thread_index)

    increment = drift + diffusion * dWt
    return increment

## Executing

### Initial Parameters

In [5]:
initial = 1.0    # initial stock price
r = 0.05         # growth rate
v = 0.20         # volatility (sigma)
dt = 1 / 365     # time step
nsteps = 365     # number of time steps
length = 1.0     # length of projection (dt*nsteps)
strike = 1.0     # K = strike price

nsim = 2**20    # number of simulations = 1_048_576

### Declaring grid

In [6]:
threads_per_block = 2**7 # = 128
blocks = ceil(nsim / threads_per_block)

### GPU RNG states

In [7]:
# set RNG state for each thread with a seed
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)

### Declaring output variable

In [8]:
pv_payoffs = np.zeros(nsim, dtype=np.float32)

### Executing kernel

In [9]:
%%time
calculate_asian_payoff_gpu[blocks, threads_per_block](
    initial,
    r,
    v,
    dt,
    strike,
    nsteps,
    length,
    rng_states,
    pv_payoffs,
)

CPU times: user 459 ms, sys: 0 ns, total: 459 ms
Wall time: 459 ms


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [10]:
epv_payoff = pv_payoffs.mean()

In [11]:
print(epv_payoff)

0.057646543


### Executing CPU version

In [14]:
%%time
epv_payoff_cpu = calculate_asian_payoff_cpu(initial, r, v, dt, strike, nsim, nsteps, length)

CPU times: user 8.19 s, sys: 3.13 s, total: 11.3 s
Wall time: 11.3 s


In [15]:
epv_payoff_cpu

0.05767875469304373